# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("../output_data/cities.csv")
vacation_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Shimoda,20,JP,1587255042,63,34.67,138.95,64.40,5.82
1,1,Bluff,100,NZ,1587255043,64,-46.60,168.33,55.99,11.99
2,2,Dikson,100,RU,1587254781,87,73.51,80.55,35.65,20.71
3,3,Torrington,80,US,1587255046,89,41.80,-73.12,39.00,4.07
4,4,Jamestown,1,US,1587254722,41,42.10,-79.24,45.00,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = vacation_df[["Lat", "Lng"]]
rating = vacation_df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Use loc with condition
criteria_df = vacation_df.loc[(vacation_df['Max Temp'] < 80)]
criteria_df = criteria_df.loc[(criteria_df['Max Temp'] > 70)]
criteria_df = criteria_df.loc[(criteria_df['Wind Speed'] < 10)]
criteria_df = criteria_df.loc[(criteria_df['Cloudiness'] == 0)]
# criteria_df = criteria_df.dropna
criteria_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
209,209,Bānapur,0,IN,1587255288,83,19.78,85.18,78.75,6.11
307,307,Ladário,0,BR,1587255395,56,-19.00,-57.60,71.55,2.51
312,312,Thakurgaon,0,BD,1587255407,75,26.03,88.47,73.09,8.50
350,350,San Miguel de Tucumán,0,AR,1587255257,56,-26.82,-65.22,73.40,4.70
431,431,San Juan,0,AR,1587255453,30,-31.54,-68.54,75.94,1.01
441,441,Narasannapeta,0,IN,1587255557,88,18.42,84.05,78.48,6.67
475,475,Yanam,0,IN,1587255595,88,16.73,82.22,79.92,6.11
529,529,Salem,0,IN,1587255525,67,11.65,78.17,77.97,1.90


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = criteria_df
#print(hotel_df["Lat"])
#print(hotel_df["Lat"].map(str))
#New columng "Coordinates" to join Lat and Long together; easier when pulling from API
#List to hold the API search results
hotel_df['Hotel Name'] = ""
#print(hotel_df.head())
# Define the search parameters
target_type = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
search_radius = 5000
params = {"keyword" : target_type,
         "radius" : search_radius,
         "key" : g_key}

# Use a for loop to retrieve data and put into hotel_result
#index = row, always true for this loop
for index, row in hotel_df.iterrows():
    #print(index)
    #print(row)
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    #print(params)
    nearest_hotel = requests.get(base_url, params = params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = nearest_hotel["results"][0]["name"]
    except:
        print("Missing... skipping")
    #hotel_result.append(nearest_hotel["results"][0]["name"])
    #print(json.dumps(nearest_hotel, indent=4))
    
hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
209,209,Bānapur,0,IN,1587255288,83,19.78,85.18,78.75,6.11,SPOT ON 49159 Hotel Digantika
307,307,Ladário,0,BR,1587255395,56,-19.00,-57.60,71.55,2.51,Pousada Corumbá Center
312,312,Thakurgaon,0,BD,1587255407,75,26.03,88.47,73.09,8.50,QURAISHI Catrabas
350,350,San Miguel de Tucumán,0,AR,1587255257,56,-26.82,-65.22,73.40,4.70,Amerian Tucuman Apart & Suites
431,431,San Juan,0,AR,1587255453,30,-31.54,-68.54,75.94,1.01,Hotel Viñas del Sol
441,441,Narasannapeta,0,IN,1587255557,88,18.42,84.05,78.48,6.67,Sri laxmi lodge
475,475,Yanam,0,IN,1587255595,88,16.73,82.22,79.92,6.11,The Regency Hotel
529,529,Salem,0,IN,1587255525,67,11.65,78.17,77.97,1.90,Sivaraj Holiday Inn


In [7]:

hotel_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
209,209,Bānapur,0,IN,1587255288,83,19.78,85.18,78.75,6.11,SPOT ON 49159 Hotel Digantika
307,307,Ladário,0,BR,1587255395,56,-19.00,-57.60,71.55,2.51,Pousada Corumbá Center
312,312,Thakurgaon,0,BD,1587255407,75,26.03,88.47,73.09,8.50,QURAISHI Catrabas
350,350,San Miguel de Tucumán,0,AR,1587255257,56,-26.82,-65.22,73.40,4.70,Catalinas Park
431,431,San Juan,0,AR,1587255453,30,-31.54,-68.54,75.94,1.01,Hotel Viñas del Sol
441,441,Narasannapeta,0,IN,1587255557,88,18.42,84.05,78.48,6.67,Sri laxmi lodge
475,475,Yanam,0,IN,1587255595,88,16.73,82.22,79.92,6.11,The Regency Hotel
529,529,Salem,0,IN,1587255525,67,11.65,78.17,77.97,1.90,Sivaraj Holiday Inn


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))